In [ ]:
#REFERENCES
# https://mathworld.wolfram.com/GnomonicProjection.html
# https://stackoverflow.com/questions/6671183/calculate-the-center-point-of-multiple-latitude-longitude-coordinate-pairs
# https://www.movable-type.co.uk/scripts/latlong.html
# https://gis.stackexchange.com/questions/143789/how-do-i-calculate-the-latitude-of-an-end-point-given-a-start-point-bearing-an
# https://www.geeksforgeeks.org/convex-hull-using-graham-scan/
# https://math.stackexchange.com/questions/402799/intersection-of-two-arcs-on-sphere
# https://blog.mbedded.ninja/mathematics/geometry/spherical-geometry/finding-the-intersection-of-two-arcs-that-lie-on-a-sphere/
#https://stackoverflow.com/questions/9678624/convex-hull-of-longitude-latitude-points-on-the-surface-of-a-sphere

In [ ]:
import math
import functools

rads = math.pi / 180
eRadius = 6371 #km

In [ ]:
#rounds near 0 to and 0 and near -1 or 1 to the extreme
def round(v, epislon=0.00001):
    if abs(v) < epislon:
        return 0
    if v + epislon > 1:
        return 1
    if v - epislon < -1:
        return -1
    return v

In [ ]:
#round floats to nearest whole number if within epislon
def roundFloat(v, epislon=0.00001):
    sign = 1 if v > 0 else -1
    absV = abs(v)
    floorV = math.floor(absV)
    if floorV < math.floor(absV + epislon):
        return sign * math.floor(absV + epislon)
    elif floorV > math.floor(absV - epislon):
        return sign * floorV
    return v

In [ ]:
#Converts a [lat,lon] to an [X,Y,Z]
def toXYZ(latLon):
    rLat,rLon = latLon[0] * rads, latLon[1] *rads

    cLat = round(math.cos(rLat))

    X = cLat * round(math.cos(rLon))
    Y = cLat * round(math.sin(rLon))
    Z = round(math.sin(rLat))
    return [X,Y,Z]

In [ ]:
def checkPlane(plane,vector):
    return round(plane[0] * vector[0] + plane[1] * vector[1] + plane[2] * vector[2]) > 0

In [ ]:
def toLatLon(xyz):
    lat = math.atan2(xyz[2], math.sqrt((xyz[0] * xyz[0] + xyz[1] * xyz[1]))) / rads
    lon = math.atan2(xyz[1],xyz[0]) / rads
    return [lat, lon]

In [ ]:
def fixLon(lon):
    if lon < -180:
        return fixLon(lon + 360)
    elif lon > 180:
        return fixLon(lon - 360)
    return lon

In [ ]:
def haversine(start, stop):
    startLat = start[0] * rads
    startLon = start[1] * rads

    stopLat = stop[0] * rads
    stopLon = stop[1] * rads

    latDiff = stopLat - startLat
    lonDiff = stopLon - startLon

    sinLatDiff = math.sin(latDiff / 2)
    sinLonDiff = math.sin(lonDiff / 2)

    a = sinLatDiff * sinLatDiff + math.cos(startLat) * math.cos(stopLat) * sinLonDiff * sinLonDiff

    c = 2 * math.atan2( math.sqrt(a), math.sqrt((1-a))) #angular distance
    return c * eRadius

    

In [ ]:
def bearing(start,stop):
    startLat = start[0] * rads
    startLon = start[1] * rads

    stopLat = stop[0] * rads
    stopLon = stop[1] * rads

    lonDiff = stopLon - startLon

    return math.atan2(
        math.sin(lonDiff) * math.cos(stopLat),
        math.cos(startLat) * math.sin(stopLat) - math.sin(startLat) * math.cos(stopLat) * math.cos(lonDiff)
    )


In [ ]:
def destination(latLon,d,b):
    lat = latLon[0] * rads
    lon = latLon[1] * rads

    a = d / eRadius

    lat2 = math.asin(math.sin(lat) * math.cos(a) + math.cos(lat) * math.sin(a) * math.cos(b))
    lon2 = lon + math.atan2(math.sin(b) * math.sin(a) * math.cos(lat), math.cos(a) - math.sin(lat) * math.sin(lat2))
    return [roundFloat(lat2 / rads), fixLon(roundFloat(lon2 / rads))]

In [ ]:
def toGnomonic(latLon, cLatLon):
    lat = latLon[0] * rads
    lon = latLon[1] * rads

    cLat = cLatLon[0] * rads
    cLon = cLatLon[1] * rads

    ccLat = math.cos(cLat)
    sLat = math.sin(lat)
    scLat = math.sin(cLat)
    cosLat = math.cos(lat)
    lonDelta = lon - cLon
    cosLonDelta = math.cos(lonDelta)

    cosc = scLat * sLat + ccLat * cosLat * cosLonDelta
    x = cosLat * math.sin(lonDelta) / cosc
    y = ( ccLat * sLat - scLat * cosLat * cosLonDelta ) / cosc

    return [x,y] 

In [ ]:
def fromGnomonic(xy, cLatLon):
    p = (xy[0] * xy[0] + xy[1] * xy[1]) ** (1/2)
    if p == 0:
        return cLatLon
    c = math.atan(p)

    cLat = cLatLon[0] * rads
    cLon = cLatLon[1] * rads

    cosc = math.cos(c)
    sLat = math.sin(cLat)
    sinc = math.sin(c)
    cosLat =  math.cos(cLat)

    lat = math.asin(cosc * sLat + xy[1] * sinc * cosLat / p) 
    lon = cLon + math.atan( xy[0] * sinc / ( p * cosLat * cosc - xy[1] * sLat * sinc ) ) 

    return [roundFloat(lat / rads) , fixLon(roundFloat(lon / rads))]

In [ ]:
def dot(a,b):
    return a[0] * b[0] + a[1] * b[1] + a[2] * b[2]

In [ ]:
def cross(a,b):
    return [ 
        a[1]*b[2] - a[2]*b[1],
        a[2]*b[0] - a[0]*b[2],
        a[0]*b[1] - a[1]*b[0]
    ]

In [ ]:
def split(start,stop):
    xyzStart = toXYZ(start)
    xyzStop = toXYZ(stop)
    startToStop = cross(xyzStart,xyzStop)

    startSplit = toXYZ([10,180])
    endSplit = toXYZ([0,180])
    splitCross = cross(startSplit, endSplit)

    intersection = cross(splitCross,startToStop)

    return toLatLon(intersection)

In [ ]:
def getNorm(a):
    return math.sqrt(a[0] * a[0] + a[1] * a[1] + a[2] * a[2])

In [ ]:
#can be used to check if split intersection is valid 
def checkIntersection(start,stop,res):
    a1 = math.acos(dot(start,res) / (getNorm(start) * getNorm(res)) )
    a2 = math.acos(dot(res,stop) / (getNorm(stop) * getNorm(res)) )

    intial = math.acos(dot(start,stop) / ( getNorm(start) * getNorm(stop) ))
    if round(a1 + a2 - intial) == 0:
        return True
    return False


In [ ]:
#def convexHull3D(latLonArray):
    #convert to xyz
    #find center xyz
    #filter to xyz's above the plane created by origin to center vector
    #we need the lat lon points that pass this test
    #take lat,lons and gnomonic project based on center
    #hull these points
    #reproject hull points
    #slice into two groups for easier handling of anti-merdian

In [ ]:
def getCenter(points):
    center = [0,0,0]
    numPoints = len(points)
    for p in points:
        xyz = toXYZ(p)
        center[0] += xyz[0]
        center[1] += xyz[1]
        center[2] += xyz[2]
    center[0] = center[0] / numPoints
    center[1] = center[1] / numPoints
    center[2] = center[2] / numPoints
    return toLatLon(center)

In [ ]:
def polarAngleCompare(p,q,r):
    rAngle = math.atan((r[1] - p[1]) /  (r[0] - p[0]) ) if r[0] - p[0] != 0 else 0
    qAngle = math.atan((q[1] - p[1]) / (q[0] - p[0]) ) if q[0] - p[0] != 0 else 0

    #print(f"p {p} qAngle {qAngle} q {q} rAngle {rAngle} r {r}")
    if rAngle > qAngle:
        return -1
    elif qAngle > rAngle:
        return 1
    return 0

In [ ]:
def polarTieBreak(p,q,r,res):
    if res != 0:
        return res
    rDistance = (r[0] - p[0]) * (r[0] - p[0]) + (r[1] - p[1]) * (r[1] - p[1])
    qDistance = (q[0] - p[0]) * (q[0] - p[0]) + (q[1] - p[1]) * (q[1] - p[1])
    if rDistance >= qDistance:
        return -1
    else:
        return 1

In [ ]:
def orientation(p,q,r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0: #collinear
        return 0
    elif val > 0: #clock wise
        return 1
    else: #counterclock wise
        return 2

In [ ]:
def getSortCriteria(p):
    return lambda x,y: polarTieBreak(p,x,y,polarAngleCompare(p,x,y))



In [ ]:
#WORK IN PROGRESS

def grahamScan(xyArray):
    #find lowest point on y if tie take fatherest left
    pMin = xyArray[0]
    minSpot = 0
    n = len(xyArray)
    for i in range(1,n):
        if xyArray[i][1] < pMin[1] or xyArray[i][1] == pMin[1] and xyArray[i][0] < pMin[0]:
            pMin = xyArray[i]
            minSpot = i

    swap = xyArray[0]
    xyArray[0] = pMin
    xyArray[minSpot] = swap
    compare = getSortCriteria(pMin)
    #print(f"presorted {xyArray}")
    #sort based on polar angle from min point and distance from min point
    xyArray = sorted(xyArray, key=functools.cmp_to_key(compare))
    #print(f"post sorted {xyArray}")
    m = 1
    stop = n -1
    i = 1 
    # go through and exclude points that have the same orientation
    while(i < n):
        while i < stop:
            o = orientation(pMin,xyArray[i],xyArray[i+1])
            if o == 0:
                i += 1
            else:
                break
        xyArray[m] = xyArray[i]
        i += 1
        m += 1

    if m < 3:
        return xyArray[0:m]

    s = []
    s.append(xyArray[0])
    s.append(xyArray[1])
    s.append(xyArray[2])
    sLen = len(s)
    for i in range(3,m):
        while ( (sLen > 1) and (orientation(s[sLen - 2], s[sLen -1 ], xyArray[i])) != 2):
            s.pop()
            sLen = sLen - 1
        s.append(xyArray[i])
        sLen = sLen + 1
    return s[::1]
    

In [ ]:
# d = haversine([45,45],[46,47])
# b = bearing([45,45],[46,47])
# newP = destination([45,45], 191.461, 0.93876502)
# print(f"distance {d} bearing {b} point {newP}")

In [ ]:
# xy = toGnomonic([47,100],[60,60])
# latLon = fromGnomonic(xy,[60,60])
# print(f"lat {latLon[0]} lon {latLon[1]}")

In [ ]:
# xyz = toXYZ([45,45])
# lat,lon = toLatLon(xyz)
# print(f"lat {lat} lon {lon}")

In [ ]:
# plane = toXYZ([89.99999,0])
# print(checkPlane(plane,toXYZ([50,-40])))

In [ ]:
# split([60,170],[60,-170])

In [ ]:
# p = [[63,-167],[55,155],[70,170],[65,160],[67,-172],[60,175]]
# c = getCenter(p)
# p = [toGnomonic(newP, c) for newP in p]
# p = grahamScan(p)
# p = [fromGnomonic(newP,c) for newP in p]
# p

In [ ]:
# p = [[-5,55],[7,60],[3,45],[-10,57.6],[-3,72]]
# p = [toGnomonic(newP, [5,60]) for newP in p]
# p = grahamScan(p)
# p = [fromGnomonic(newP,[5,60]) for newP in p]
# p

In [ ]:
# p = [[-5,-5],[7,6],[3,4],[-10,5.6],[-3,-7]]
# p = [toGnomonic(newP, [5,60]) for newP in p]
# p = grahamScan(p)
# p = [fromGnomonic(newP,[5,60]) for newP in p]
# p

In [ ]:
# p = [[-5,-5],[3,-5],[7,6],[3,4],[-10,5.6],[-3,-7]]
# c = getCenter(p)
# p = [toGnomonic(newP,c) for newP in p]
#p = [[-5,-5],[3,-5],[7,6],[3,4],[-10,5.6],[-3,-7]]
# p = grahamScan(p)
#p = [fromGnomonic(newP,c) for newP in p]
# p